<a href="https://colab.research.google.com/github/parvathysarat/gpt2-text-generation/blob/master/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install transformers
!git clone https://github.com/huggingface/transformers
os.chdir('transformers')
print(os.getcwd())
!pip install .

     |████████████████████████████████| 778kB 4.9MB/s 
     |████████████████████████████████| 1.1MB 28.3MB/s 
     |████████████████████████████████| 3.0MB 40.0MB/s 
     |████████████████████████████████| 890kB 54.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=c97418360ffe92bf911645ceec1bd812d26b7d58221b42de3e7e9195d35edd1a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Cloning into 'transformers'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 38349 (delta 9), reused 9 (delta 2), pack-reused 38327
Receiving objects: 100% (38349/38349), 27.74 MiB | 26.13 MiB/s, done.
Resolving deltas: 100% (26569/26569), done.


In [13]:
import os
os.chdir('examples/language-modeling/')

## Fine tuning GPT-2 language model on our dataset

Add to transformers/example/language-modeling/run-language-modelling.py file the following snippet : <br>

```
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
```


In [37]:
N=1
try:os.mkdir('../../../model')
except:pass
OUTPUT_DIR='../../../model/'
print(os.getcwd())
TRAIN_FILE='../../../dataset/train.txt'
VALID_FILE='../../../dataset/valid.txt'


/content/transformers/examples/language-modeling


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]= "1"
!python run_language_modeling.py \
--output_dir=$OUTPUT_DIR \
--model_type=gpt2 \
--model_name_or_path=gpt2 \
--do_train \
--train_data_file=$TRAIN_FILE \
--do_eval \
--eval_data_file=$VALID_FILE \
--per_device_train_batch_size=2 \
--per_device_eval_batch_size=2 \
--line_by_line \
--evaluate_during_training \
--learning_rate=5e-5 \
--num_train_epochs=5 \
--overwrite_output_dir

Streaming output truncated to the last 5000 lines.
Iteration:  46% 629/1376 [19:20<20:56,  1.68s/it]
Iteration:  46% 630/1376 [19:21<20:27,  1.65s/it]
Iteration:  46% 631/1376 [19:23<19:23,  1.56s/it]
Iteration:  46% 632/1376 [19:24<19:33,  1.58s/it]
Iteration:  46% 633/1376 [19:26<21:00,  1.70s/it]
Iteration:  46% 634/1376 [19:28<21:27,  1.73s/it]
Iteration:  46% 635/1376 [19:30<20:19,  1.65s/it]
Iteration:  46% 636/1376 [19:31<18:52,  1.53s/it]
Iteration:  46% 637/1376 [19:32<18:46,  1.52s/it]
Iteration:  46% 638/1376 [19:34<19:18,  1.57s/it]
Iteration:  46% 639/1376 [19:35<18:35,  1.51s/it]
Iteration:  47% 640/1376 [19:37<17:51,  1.46s/it]
Iteration:  47% 641/1376 [19:39<18:43,  1.53s/it]
Iteration:  47% 642/1376 [19:41<22:02,  1.80s/it]
Iteration:  47% 643/1376 [19:42<20:17,  1.66s/it]
Iteration:  47% 644/1376 [19:44<19:55,  1.63s/it]
Iteration:  47% 645/1376 [19:45<19:50,  1.63s/it]
Iteration:  47% 646/1376 [19:47<20:54,  1.72s/it]
Iteration:  47% 647/1376 [19:49<19:20,  1.59s/it]

## Evaluate perplexity on test set

In [ ]:
TEST_FILE='../../../dataset/valid.txt'
CUDA_VISIBLE_DEVICES="1" python run_language_modeling.py \
--output_dir=$OUTPUT_DIR \
--model_type=gpt2 \
--model_name_or_path=$OUTPUT_DIR \
--do_eval \
--eval_data_file=$TEST_FILE \
--per_device_eval_batch_size=2 \
--line_by_line

## Text generation

Setting prompt text: "I always wonder"

Using top-k sampling decoder from GPT-2 (k=50 has been proven to be more effective at generating non-repetitive texts.) Other decoders are also available.

In [ ]:
prompt_text = "I always wonder"
encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=True, return_tensors=”pt”)

In [ ]:
os.chdir('../text-generation')
K=50
CUDA_VISIBLE_DEVICES="1" python run_generation.py \
--model_type gpt2 \
--model_name_or_path $OUTPUT_DIR \
--length 300 \
--prompt "<BOS>" \
--stop_token "<EOS>" \
--k $K \
--num_return_sequences 5